## procedure: 
1. check for 3991 primary accessions in UP5640 file from UniProt- True  = 5988
2. grabbed the accession column from 5988 and check number of unique accessions in file = 3903, 88 IDs were not in primary accession form in the proteome file
    - reasons could be that 1. only isoform version of ID in file 2. accession not apart of proteome set, only in UNIPROTKB (my CCDS set is not proteome filtered)
    
```python
print(len(acc)) # 3991 
print(len(IDlist)) # 3903

# difference of 88 (3991 - 3903)
```

3. saved file of accessions not found in the bed file "UniProtKB_entries_canonicalID_notFoundinBedProteome5640_total88.csv" and moving on with the 3903 unique IDs that are in the bed file as the canonical ID format (missing a '-#' after entry

4. simplified df of 5988 rows to only these columns ['chr', 'UniProtKB_accession','strand', 'thick_start', 'thick_end', 'aminoacids']

5. after column filtering, i dropped all redundant rows: basically the redundancy caused by transcription start site positions are not relevant to me.  only care about thick start and end which is cds positions

```python 
bedTsimple.drop_duplicates(subset=None, keep='first', inplace=True)
bedTsimple.shape # (4957, 6)
len(bedTsimple.UniProtKB_accession.unique()) # 3903
```

6. simplified again the dataframe down to only chr and accession. there is redundancy of accessions with only step 4 columns do to differences in thick start and end, strand...DROPPED the redundancy by keepign only the first unique instance

```python
# simplify down to chromosome and ID df (for searching dbNSFP chr parsed files)
CHRID = bedTsimple[['chr','UniProtKB_accession']].copy()
CHRID.drop_duplicates(subset=None, keep='first', inplace=True)
```

7. expected only chr and accession df will have all unique accession but this was not the case. 2 accessions had chr multi mapping problem. I dropped these accessions to make dbNSFP coordinate mapping easier. 

```bash 
chr	UniProtKB_accession
chr1	P62805
chr12	P62805
chr6	P62805
chr1	P84243
chr17	P84243
# post drop 
len(CHR2.UniProtKB_accession.unique())
# 3901 unique accessions

```

8. saved chr and accession file with all unique 3901 accessions as "UniProtKB_proteome_accessions_inBedfile_labeled3991set_chr_ID_simple_3901.csv"

9. how many IDs are linked to each chromosome? 

```python 
bedCHR = CHR2.chr.value_counts()
bedCHR.to_csv("UniProtKB_proteome_accession_total3901unique_chr_valueCounts.csv")
```

10. filtered out the 2 IDs with multi chr mapping in the original simplified column dataframe of length  (4957, 6)

```python 
bed1 = bedTsimple[bedTsimple['UniProtKB_accession'] != 'P62805']
bed2 = bed1[bed1['UniProtKB_accession'] != 'P84243']
bed2.shape # 4931,6 , difference of 26 rows
len(bed2.UniProtKB_accession.unique()) # 3901
```

11. using df of 4931 rows 3901 unique accessions, i found all duplicate rows based on accession grouping, and saved this file. These IDs may be trickier to confirm dbNSFP matching canonical.

```python
dupes = pd.concat(d for _, d in bed2.groupby("UniProtKB_accession") if len(d) > 1)
# shape is 1581 rows × 6 columns
dupes.to_csv("duplicated_IDs_from_3901uniqueIDs_bedfile_4931rows_1581duplicatedIDrows.csv",index=False)
```

12. SAVED THE bed file derived df of 4931 rows with 3901 unique accessions, this is the source of the duplicate file saved above. "UniProtKB_proteome_accessions_labeledBefore3991_total3901uniqueIDs_4931rows.csv"

13. saved all unique 3901 primary accessions with chromosome annotation as simple file UniProtKB_proteome_accessions_inBedfile_labeled3991set_chr_ID_simple_3901.csv" and even simplier version with only primary accessions as "only_3901_accessions.csv"

14. split accessions into chr files to make search dbNSFP files faster

```python
CHR = pd.read_csv("UniProtKB_proteome_accessions_inBedfile_labeled3991set_chr_ID_simple_3901.csv")

chrlist = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22','chrX','chrY']

for i in chrlist:
    ch = str(i)
    df = CHR2[CHR2['chr'] == ch].copy()
    df.drop(['chr'],inplace=True,axis=1)
    df.to_csv(ch+"_only_accessions_from3901_proteome5640bed.csv",index=False)
```

15. removed the header of all chr files made above, removed newline at end of file (which makes wc -l counts 1 less than actual file line count since unix requires newline char for new line to count

16. on hoffman, used these no header, primary accessions files to search each dbNSFP4.0avariant_chr... file

```bash
LC_ALL=C grep -F -f chr1_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr1 > fgrep_chr1.txt
LC_ALL=C grep -F -f chr2_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr2 > fgrep_chr2.txt
LC_ALL=C grep -F -f chr3_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr3 > fgrep_chr3.txt
LC_ALL=C grep -F -f chr4_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr4 > fgrep_chr4.txt
LC_ALL=C grep -F -f chr5_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr5 > fgrep_chr5.txt
LC_ALL=C grep -F -f chr6_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr6 > fgrep_chr6.txt
LC_ALL=C grep -F -f chr7_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr7 > fgrep_chr7.txt
LC_ALL=C grep -F -f chr8_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr8 > fgrep_chr8.txt
LC_ALL=C grep -F -f chr9_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr9 > fgrep_chr9.txt
LC_ALL=C grep -F -f chr10_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr10 > fgrep_chr10.txt
LC_ALL=C grep -F -f chr11_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr11 > fgrep_chr11.txt
LC_ALL=C grep -F -f chr12_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr12 > fgrep_chr12.txt
LC_ALL=C grep -F -f chr13_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr13 > fgrep_chr13.txt
LC_ALL=C grep -F -f chr14_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr14 > fgrep_chr14.txt
LC_ALL=C grep -F -f chr15_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr15 > fgrep_chr15.txt
LC_ALL=C grep -F -f chr16_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr16 > fgrep_chr16.txt
LC_ALL=C grep -F -f chr17_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr17 > fgrep_chr17.txt
LC_ALL=C grep -F -f chr18_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr18 > fgrep_chr18.txt
LC_ALL=C grep -F -f chr19_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr19 > fgrep_chr19.txt
LC_ALL=C grep -F -f chr20_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr20 > fgrep_chr20.txt
LC_ALL=C grep -F -f chr21_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr21 > fgrep_chr21.txt
LC_ALL=C grep -F -f chr22_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chr22 > fgrep_chr22.txt
LC_ALL=C grep -F -f chrX_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chrX > fgrep_chrX.txt
LC_ALL=C grep -F -f chrY_only_accessions_from3901_proteome5640bed.csv dbNSFP4.0a_variant.chrY > fgrep_chrY.txt
```

## dbNSFP4.0a file with all chr - filter for only lines that contain uniprot accession in 3901 file (created in python code below)


### I could also feed in 3991 to test if they are in the dbNSFP file!!

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import csv

In [2]:
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000

# makes display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:90% !important;}</style>"))

In [3]:
sys.path.append("/Users/mariapalafox/Desktop/Toolbox/")
from all_funx import *

In [4]:
os.chdir("/Users/mariapalafox/Box Sync/CODE_DATA/dir_MAPpaper/COORDINATES/")
print(os.listdir())

['Cys_ever_labeled_set_6315.csv', 'only_3901_accessions.csv', 'all3991_everLabeled_entries_simple.csv', 'duplicated_IDs_from_3901uniqueIDs_bedfile_4931rows_1581duplicatedIDrows.csv', 'UniProtKB_proteome_accessions_inBedfile_labeled3991set_chr_ID_simple_3901.csv', 'UniProtKB_proteome_accessions_labeledBefore3991_total3901uniqueIDs_4931rows.csv', 'UP000005640_9606_proteome.bed', 'UniProtKB_proteome_accession_total3901unique_chr_valueCounts.csv', 'chr1_only_accessions_from3901_392.csv', 'Lys_ever_labeled_set_9162.csv', 'UniProtKB_entries_canonicalID_notFoundinBedProteome5640_total88.csv']


In [5]:
bed = pd.read_csv("UP000005640_9606_proteome.bed",sep='\t',header=None)

In [6]:
describeMe(bed)

(114370, 14)
Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], dtype='int64')
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
dtype: int64


In [7]:
colnames = ['chr','start','end','UniProtKB_accession','score_0default','strand','thick_start','thick_end','anno_color','number_blocks','block_sizes','block_starts','accession_proteome','aminoacids']
print(len(colnames))

14


## from README for proteome bed file

2. UniProt Proteome ID_Taxonomy ID_UniProtKB Annotation Type.bed
    
```bash
A BED detail formatted tab delimited file containing
- Chromosome name.
- Annotation start coordinate on the chromosome.
- Annotation end coordinate on the chromosome.
- UniProtKB accession, BED line name.
- Score set to 0 as default.
- DNA strand +/- for forward or reverse.
- Thick start coordinate on the chromosome.
- Thick end coordinate on the chromosome.
- Annotation color (RGB).
- Number of blocks representing the annotation.
- Block sizes, a comma separated list.
- Block starts, a comma separated list of block offsets relative to the
annotation start.
- Annotation identifier. (accession in proteome file)
- Annotation description, a semi-colon (;) separated list that can consist of:
  1. amino acid or amino acid range the UniProt annotation covers or amino acid
change (variants only).
  2. annotation description
  3. disease name and OMIM identifier (variants only)
  4. PubMed literature evidence
if available. This column has a maximum of 254 characters.

Missing values are represented by dots.
```

In [8]:
bed.columns = colnames
bed.head(10)

,chr,start,end,UniProtKB_accession,score_0default,strand,thick_start,thick_end,anno_color,number_blocks,block_sizes,block_starts,accession_proteome,aminoacids
0,chr1,69054,70108,Q8NH21,0,+,69090,70007,"255,0,0",1,1054,0,Q8NH21,M1-F305
1,chr1,450702,451697,A0A126GV92,0,-,450721,450721,"255,0,0",1,995,0,A0A126GV92,M1-S312
2,chr1,450702,451697,Q6IEY1,0,-,450721,450721,"255,0,0",1,995,0,Q6IEY1,M1-S312
3,chr1,685678,686673,A0A126GV92,0,-,685697,685697,"255,0,0",1,995,0,A0A126GV92,M1-S312
4,chr1,685678,686673,Q6IEY1,0,-,685697,685697,"255,0,0",1,995,0,Q6IEY1,M1-S312
5,chr1,923927,939291,A6PWC8,0,+,924431,939291,"255,0,0",7,"1021,92,182,51,125,90,17","0,1994,6227,7111,11844,15112,15347",A6PWC8,"M1-G173,G173-V203,N204-R264,R264-S281,S281-G32..."
6,chr1,925149,935793,Q5SV95,0,+,925941,935793,"255,0,0",5,"40,92,182,51,22","0,772,5005,5889,10622",Q5SV95,"M1-V24,N25-R85,R85-S102,S102-T109"
7,chr1,925737,944575,Q96NU1,0,+,925941,944152,"255,0,0",14,"63,92,182,51,125,90,186,163,116,79,500,125,111...","0,184,4417,5301,10034,13302,13537,15406,16398,...",Q96NU1,"M1-V24,N25-R85,R85-S102,S102-G144,G144-E174,E1..."
8,chr1,925740,944581,A0A087WX24,0,+,925941,942854,"255,0,0",12,"60,92,182,51,34,135,116,79,500,125,111,674","0,181,4414,5298,10031,15431,16395,16669,16818,...",A0A087WX24,"M1-V24,N25-R85,R85-S102,S102-S113,G114-E158,G1..."
9,chr1,925740,944581,A0A087WXB3,0,+,925941,944152,"255,0,0",11,"60,92,182,125,90,141,79,500,125,111,674","0,181,4414,10031,13299,13531,16669,16818,17512...",A0A087WXB3,"M1-V24,N25-S85,S85-G127,G127-A157,A157-G204,G2..."


In [9]:
bed.accession_proteome.equals(bed.UniProtKB_accession)
# two columns with uniprot entry are identical

True

In [10]:
labeledID = pd.read_csv("all3991_everLabeled_entries_simple.csv")
describeMe(labeledID)

(3991, 1)
Index(['Entry'], dtype='object')
Entry    0
dtype: int64


In [11]:
IDlist = labeledID.Entry.tolist()

In [12]:
bedTrue = addcolumnconditionalDrop(IDlist,bed,'UniProtKB_accession','in3991set')
checkColumnValues(bedTrue,'in3991set')

  in3991set  Count
0      True   5988


In [13]:
describeMe(bedTrue)

(5988, 15)
Index(['chr', 'start', 'end', 'UniProtKB_accession', 'score_0default',
       'strand', 'thick_start', 'thick_end', 'anno_color', 'number_blocks',
       'block_sizes', 'block_starts', 'accession_proteome', 'aminoacids',
       'in3991set'],
      dtype='object')
chr                    0
start                  0
end                    0
UniProtKB_accession    0
score_0default         0
strand                 0
thick_start            0
thick_end              0
anno_color             0
number_blocks          0
block_sizes            0
block_starts           0
accession_proteome     0
aminoacids             0
in3991set              0
dtype: int64


In [14]:
accessions = bedTrue.UniProtKB_accession.tolist()
# how many unique IDs are in the bed file out of 3991

set_accessions = set(accessions)
acc = list(set_accessions)

In [16]:
print(len(acc))
print(len(IDlist))

3903
3991


In [19]:
3991 - 3903

88

In [17]:
notinbed = list(set(IDlist) - set(acc))

In [20]:
notbed = pd.DataFrame(notinbed)
notbed.columns = ['entry_notinbed_proteome']
notbed.head(10)

,entry_notinbed_proteome
0,P11182
1,Q86UQ4
2,Q12912
3,Q8IWI9
4,Q03001
5,O75150
6,Q9H1A7
7,Q9NQW7
8,O14965
9,Q9UNH6


In [21]:
notbed.to_csv("UniProtKB_entries_canonicalID_notFoundinBedProteome5640_total88.csv",index=False)

## only 3903 out of 3991 uniprot IDs (the canonical version of ID with no '-#') were in proteome bed file

### lost 88 IDs

### REASON FOR NOT BEING IN BED FILE :
1. the entry is in UniProtKB but not in the Proteome set of UniProt for human
2. the entry is in bed file but has a '-#' extension which appears as not found with current function 

**moving on with the 3903 unique IDs that are in the bed file as the canonical ID (missing a '-#' after entry)**

In [22]:
bedTrue.head(5)

,chr,start,end,UniProtKB_accession,score_0default,strand,thick_start,thick_end,anno_color,number_blocks,block_sizes,block_starts,accession_proteome,aminoacids,in3991set
0,chr1,1013422,1014540,P05161,0,+,1013573,1014477,"255,0,0",2,"154,557","0,561",P05161,"M1-M1,G2-S165",True
1,chr1,1216907,1232001,Q9BRK5,0,-,1217060,1228620,"255,0,0",7,"781,176,159,114,137,479,110","0,1550,1861,6336,6924,11560,14984",Q9BRK5,"M1-K109,K109-G155,G155-T193,T193-D246,D246-E30...",True
2,chr1,1324766,1328897,Q5TA50,0,+,1326910,1327762,"255,0,0",3,"336,197,1657","0,2069,2474",Q5TA50,"M1-R41,R41-P214",True
3,chr1,1401907,1407313,Q9BYC9,0,-,1402003,1407226,"255,0,0",4,"349,78,111,183","0,3901,5001,5223",Q9BYC9,"M1-R29,H30-T66,L67-K92,C93-H149",True
4,chr1,1541672,1574869,Q9NP77,0,-,1541984,1574789,"255,0,0",5,"495,119,140,144,392","0,2196,3190,23100,32805",Q9NP77,"M1-S27,S27-L75,L75-D122,D122-C161,I162-Y194",True


In [42]:
keepme = ['chr', 'UniProtKB_accession','strand', 'thick_start', 'thick_end', 'aminoacids']
bedTsimple = bedTrue[keepme].copy()

# thick start and thick end are the cds positions (not the transcription start site positions)

### dropping duplicated rows in file then parsing by chr

In [43]:
bedTsimple.drop_duplicates(subset=None, keep='first', inplace=True)
bedTsimple.shape

(4957, 6)

In [44]:
len(bedTsimple.UniProtKB_accession.unique())

3903

In [48]:
# simplify down to chromosome and ID df (for searching dbNSFP chr parsed files)
CHRID = bedTsimple[['chr','UniProtKB_accession']].copy()

,chr,UniProtKB_accession
0,chr1,P05161
1,chr1,Q9BRK5
2,chr1,Q5TA50
3,chr1,Q9BYC9
4,chr1,Q9NP77
5,chr1,O95544
7,chr1,P62873
9,chr1,Q05513
10,chr1,O15258
12,chr1,Q8N1G4


In [49]:
CHRID.drop_duplicates(subset=None, keep='first', inplace=True)
CHRID

,chr,UniProtKB_accession
0,chr1,P05161
1,chr1,Q9BRK5
2,chr1,Q5TA50
3,chr1,Q9BYC9
4,chr1,Q9NP77
5,chr1,O95544
7,chr1,P62873
9,chr1,Q05513
10,chr1,O15258
12,chr1,Q8N1G4


In [51]:
dupID = pd.concat(d for _, d in CHRID.groupby("UniProtKB_accession") if len(d) > 1)
dupID

,chr,UniProtKB_accession
293,chr1,P62805
1204,chr12,P62805
4564,chr6,P62805
508,chr1,P84243
2434,chr17,P84243


In [52]:
# dropping IDs that have multiple chr mapping
CHR1 = CHRID[CHRID['UniProtKB_accession'] != 'P62805']
CHR2 = CHR1[CHR1['UniProtKB_accession'] != 'P84243']

In [54]:
len(CHR2.UniProtKB_accession.unique())

3901

In [58]:
CHR2.head(3)

,chr,UniProtKB_accession
0,chr1,P05161
1,chr1,Q9BRK5
2,chr1,Q5TA50


In [67]:
CHR2.to_csv("UniProtKB_proteome_accessions_inBedfile_labeled3991set_chr_ID_simple_3901.csv",index=False)

In [68]:
bedCHR = CHR2.chr.value_counts()
bedCHR
bedCHR.to_csv("UniProtKB_proteome_accession_total3901unique_chr_valueCounts.csv")

In [60]:
bed1 = bedTsimple[bedTsimple['UniProtKB_accession'] != 'P62805']
bed2 = bed1[bed1['UniProtKB_accession'] != 'P84243']
bed2.shape

(4931, 6)

In [63]:
len(bed2.UniProtKB_accession.unique())

3901

In [62]:
dupes = pd.concat(d for _, d in bed2.groupby("UniProtKB_accession") if len(d) > 1)
dupes

,chr,UniProtKB_accession,strand,thick_start,thick_end,aminoacids
1448,chr12,A3KN83,-,123289108,123350441,"M1-Q44,S45-R79,Q80-T184,T184-M217,K218-L250,L2..."
1449,chr12,A3KN83,-,123295904,123350441,"M1-Q44,S45-R79,Q80-T184,T184-M217,K218-L250,L2..."
5230,chr7,A4D1F6,-,92144915,92163359,"M1-K639,L640-G706,G706-E760,E760-S799,S799-F860"
5231,chr7,A4D1F6,-,92144957,92163359,"M1-K639,L640-G706,G706-E760,E760-S799,S799-F860"
3632,chr22,A6NHG4,+,23966914,23971405,"M1-D36,R37-R95,R95-I134"
3633,chr22,A6NHG4,+,23966915,23971407,"M1-D36,R37-R95,R95-I134"
2425,chr17,A9UHW6,-,75266277,75270163,"M1-D28,D28-Q64,A65-R116,V117-E147,V148-D222"
2426,chr17,A9UHW6,-,75266284,75270166,"M1-D28,D28-Q64,A65-R116,V117-E147,V148-D222"
1838,chr15,B2RTY4,-,71822359,72045794,"M1-E280,A281-G312,G312-R333,R333-Q366,I367-P38..."
1839,chr15,B2RTY4,-,71826579,72045797,"M1-E280,A281-G312,G312-R333,R333-Q366,I367-P38..."


In [64]:
dupes.to_csv("duplicated_IDs_from_3901uniqueIDs_bedfile_4931rows_1581duplicatedIDrows.csv",index=False)

In [65]:
bed2.shape

(4931, 6)

In [69]:
bed2.to_csv("UniProtKB_proteome_accessions_labeledBefore3991_total3901uniqueIDs_4931rows.csv",index=False)

In [75]:
chr1 = CHR2[CHR2['chr'] == 'chr1']

In [76]:
chr1.shape

(392, 2)

In [77]:
chr1.drop(['chr'],inplace=True,axis=1)
chr1

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,UniProtKB_accession
0,P05161
1,Q9BRK5
2,Q5TA50
3,Q9BYC9
4,Q9NP77
5,O95544
7,P62873
9,Q05513
10,O15258
12,Q8N1G4


In [78]:
chr1.to_csv("chr1_only_accessions_from3901_392.csv",index=False)

In [80]:
CHR_ids = CHR2.drop(['chr'],axis=1)
CHR_ids

,UniProtKB_accession
0,P05161
1,Q9BRK5
2,Q5TA50
3,Q9BYC9
4,Q9NP77
5,O95544
7,P62873
9,Q05513
10,O15258
12,Q8N1G4


In [81]:
CHR_ids.to_csv("only_3901_accessions.csv",index=False)

In [5]:
CHR = pd.read_csv("UniProtKB_proteome_accessions_inBedfile_labeled3991set_chr_ID_simple_3901.csv")


In [6]:
# going to make simple chr accession file for each chr and search with fgrep
chrlist = ['chr1','chr2','chr3','chr4','chr5','chr6','chr7','chr8','chr9','chr10','chr11','chr12','chr13','chr14','chr15','chr16','chr17','chr18','chr19','chr20','chr21','chr22','chrX','chrY']

In [8]:
for i in chrlist:
    ch = str(i)
    df = CHR[CHR['chr'] == ch].copy()
    df.drop(['chr'],inplace=True,axis=1)
    df.to_csv(ch+"_only_accessions_from3901_proteome5640bed.csv",index=False)